In [ ]:
import hmac
import hashlib
import requests
from time import gmtime, strftime
import json
import pandas as pd
import numpy as np

In [ ]:
import requests
import hmac
import hashlib
from time import gmtime, strftime

REQUEST_METHOD = "GET"
DOMAIN = "https://api-gateway.coupang.com"
URL = "/v2/providers/affiliate_open_api/apis/openapi/v1/reports/orders"

# Replace with your own ACCESS_KEY, SECRET_KEY, startDate, endDate
ACCESS_KEY = access_key
SECRET_KEY = secret_key
startDate = "20240417"
endDate = "20240418"
# Assuming subId is not required, if needed add it to the query parameters

def generateHmac(method, url, secretKey, accessKey, queryString=""):
    # Correct formatting for date-time if needed
    datetimeGMT = strftime('%y%m%d', gmtime()) + 'T' + strftime('%H%M%S', gmtime()) + 'Z'
    message = datetimeGMT + method + url + queryString

    signature = hmac.new(bytes(secretKey, 'utf-8'), 
                         message.encode('utf-8'), 
                         hashlib.sha256).hexdigest()

    return "CEA algorithm=HmacSHA256, access-key={}, signed-date={}, signature={}".format(accessKey, datetimeGMT, signature)


    return "CEA algorithm=HmacSHA256, access-key={}, signed-date={}, signature={}".format(accessKey, datetimeGMT, signature)

def fetch_data(page):
    queryString = "startDate={}&endDate={}&page={}".format(startDate, endDate, page)
    fullURL = "{}{}?{}".format(DOMAIN, URL, queryString)
    authorization = generateHmac(REQUEST_METHOD, URL, SECRET_KEY, ACCESS_KEY, queryString)
    response = requests.request(method=REQUEST_METHOD, url=fullURL,
                                headers={
                                    "Authorization": authorization,
                                    "Content-Type": "application/json"
                                })
    return response.json()

def main():
    page = 1
    results = []

    while True:
        response = fetch_data(page)
        if 'data' in response:
            results.extend(response['data'])
            if len(response['data']) < 1000:
                break
            page += 1
        else:
            # Handle errors or missing 'data' key
            print(f"Error retrieving data: {response.get('message', 'No error message available')}")
            break

    print(f"Total items fetched: {len(results)}")
    return results


# Execute the main function to handle pagination
all_data = main()
